# Union Pacific Railroad: Empty car supply-demand

## Problem description
The sources of empty cars are in **Baltimore** and **Atlanta** with 200 and 300 cars respectively. 

The shipments pass through either **Chicago** and **Dallas** or there is one exception of **Baltimore** to **Fresno** direct.

The demands are at **Great Falls** (100), **Eugene** (150) and **Fresno** (200).

The costs per car for different routes are:
 - Atlanta Chicago \$72
 - Atlanta Dallas \$78
 - Baltimore Chicago \$70
 - Baltimore Dallas \$136
 - Baltimore Fresno \$277
 - Chicago Fresno \$213
 - Chicago Great Falls \$138
 - Chicago Eugene \$214
 - Dallas Eugene \$206
 - Dallas Fresno \$156
 
 What is the minimal cost solution?

In [1]:
# Define our decision variables
import pulp

SOURCES = ["Atlanta", "Baltimore"]
MIDWAYS = ["Chicago", "Dallas"]
DESTINATIONS = ["Eugene", "Fresno", "Great Falls"]
NODES = [SOURCES, MIDWAYS, DESTINATIONS]
NODES = [item for items in NODES for item in items]
ARCS = [("Atlanta", "Chicago"),
        ("Atlanta", "Dallas"),
        ("Baltimore", "Chicago"),
        ("Baltimore", "Dallas"),
        ("Baltimore", "Fresno"),
        ("Chicago", "Fresno"),
        ("Chicago", "Great Falls"),
        ("Chicago", "Eugene"),
        ("Dallas", "Eugene"),
        ("Dallas", "Fresno")]

In [2]:
NODES

supply_list = [[300, 0], [200, 0], [0, 0], [0, 0], [0, 150], [0, 200], [0, 100]]
costs_data = [72, 78, 70, 136, 277, 213, 138, 214, 206, 156]

In [3]:
# Define amount of cars that are available at each source
node_data = dict(zip(NODES, supply_list))
arc_costs = dict(zip(ARCS, costs_data))

# Split the supply/demand dictionary
(supply, demand) = pulp.splitDict(node_data)

In [4]:
# Set problem variables
prob = pulp.LpProblem("UnionPac", pulp.LpMinimize)

# create our variables
arc_vars = pulp.LpVariable.dicts("Route", ARCS, None, None, pulp.LpInteger)

# Bound our variables
for arc in ARCS:
    arc_vars[arc].bounds(0, 500)

In [5]:
# Create the objecive function
prob += pulp.lpSum([arc_vars[i]*arc_costs[i] for i in ARCS]), "The total cost of transport"

In [6]:
supply

{'Atlanta': 300,
 'Baltimore': 200,
 'Chicago': 0,
 'Dallas': 0,
 'Eugene': 0,
 'Fresno': 0,
 'Great Falls': 0}

In [7]:
demand

{'Atlanta': 0,
 'Baltimore': 0,
 'Chicago': 0,
 'Dallas': 0,
 'Eugene': 150,
 'Fresno': 200,
 'Great Falls': 100}

In [8]:
# Create all problem constraints
for node in NODES:
    prob += (supply[node] + pulp.lpSum([arc_vars[(i, j)] for (i,j) in ARCS if j == node]) >=
             demand[node] + pulp.lpSum([arc_vars[(i, j)] for (i,j) in ARCS if i == node])), "Car flow conservation in node %s"%node

In [9]:
prob.writeLP("../output/UnionPacificRailroad.lp")
prob.solve()

1

In [10]:
print("Status:", pulp.LpStatus[prob.status])

Status: Optimal


In [11]:
print("Total Cost of Transportation = ", pulp.value(prob.objective))

Total Cost of Transportation =  110200.0


In [12]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

Route_('Atlanta',_'Chicago') = 0.0
Route_('Atlanta',_'Dallas') = 300.0
Route_('Baltimore',_'Chicago') = 150.0
Route_('Baltimore',_'Dallas') = 0.0
Route_('Baltimore',_'Fresno') = 0.0
Route_('Chicago',_'Eugene') = 50.0
Route_('Chicago',_'Fresno') = 0.0
Route_('Chicago',_'Great_Falls') = 100.0
Route_('Dallas',_'Eugene') = 100.0
Route_('Dallas',_'Fresno') = 200.0
